In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Riva ASR Custom Vocabulary and Lexicon Examples

This notebook walks you through the process of customizing Riva ASR vocabulary and lexicon, in order to improve Riva vocabulary coverage and recognition of difficult words, such as acronyms.

## Overview

The Flashlight decoder, deployed by default in Riva, is a lexicon-based decoder and only emits words that are present in the provided lexicon file. That means, uncommon and new words, such as domain specific terminologies, that are not present in the lexicon file, will have no chance of being generated.

On the other hand, the greedy decoder (available during the `riva-build` process with the flag `--decoder_type=greedy`) is not lexicon-based and hence can virtually produce any word or character sequence.

### Terminologies
- **Vocabulary file**: The vocabulary file is a flat text file containing a list of vocabulary words, each on its own line. For example:
```
the
i
to
and
a
you
of
that
...
```

This file is used by the `riva-build` process to generate the lexicon file. 


- **Lexicon file**: The lexicon file is a flat text file that contains the mapping of each vocabulary word to its tokenized form, e.g, sentencepiece tokens, separated by a `tab`. Below is an example:

```
with    ▁with
not     ▁not
this    ▁this
just    ▁just
my      ▁my
as      ▁as
don't   ▁don ' t
```

*Note: Ultimately, the Riva decoder makes use of the lexicon file directly (but not the vocabulary file).*

The Riva Service Maker automatically tokenizes the words in the vocabulary file to generate the lexicon file, by using the correct tokenizer that is packaged together with the acoustic model in the .riva file. By default, Riva generates 1 tokenized form for each word in the vocabulary file. 

## What can be customized?

Both the vocabulary and the lexicon files can be customized.

- Extending vocabulary enriches Riva default vocabulary, providing additional coverage for out-of-vocabulary words, terminologies and abbreviations etc...

- Customizing the lexicon file can further enrich Riva knowledge base, by providing one or more explicit pronunciations, in the form of tokenized sequences.

## Extending the vocabulary

Extending the vocabulary must be done at Riva **build** time.

When building a Riva ASR pipeline: Pass the extended vocabulary file to the `--decoding_vocab=<vocabulary_file>` parameter of the  build command. For example, the build command for the Citrinet model:

```
    riva-build speech_recognition \
   <rmir_filename>:<key> <riva_filename>:<key> \
   --name=citrinet-1024-english-asr-streaming \
   --decoding_language_model_binary=<lm_binary> \
   --decoding_vocab=<vocabulary_file> \
   --language_code=en-US \
   <other_parameters>...
```

Refer to Riva [documentation](https://docs.nvidia.com/deeplearning/riva/user-guide/docs/service-asr.html#pipeline-configuration) for build commands for supported models.


### How to modify vocabulary file

You can either provide your own vocabulary file, or extend Riva's default vocabulary file.

- BYO vocabulary file: provide a flat text file containing a list of vocabuliary words, each on its own line. Note that this file must not only contain a small list of "difficult words", but must contains all the words that you want the ASR pipeline to be able to generate, that is, including all common words.

- Modifying an existing one: Out of the box vocabulary files  for Riva supported languages can be found on NGC, for example, for English, the vocabulary file named `flashlight_decoder_vocab.txt` can be found at this [link](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/speechtotext_en_us_lm/files?version=deployable_v1.1). Alternatively, it can also be found in a deployed Riva ASR pipeline, for example, under `/data/models/citrinet-1024-en-US-asr-offline-ctc-decoder-cpu-streaming-offline/1/dict_vocab.txt` in the Riva server docker container.
You can extend this default vocabulary file with the words of interest.

Once modified, you'll have to redeploy the Riva ASR pipeline with `riva-build` while passing the flag `--decoding_vocab=<modified_vocabulary_file>`.

## Customizing the lexicon

The lexicon file that is used by the Flashlight decoder can be found in the Riva server docker container, under the Triton model folder, for example `/data/models/citrinet-1024-en-US-asr-offline-ctc-decoder-cpu-streaming-offline/1/lexicon.txt`.

Note: the actual location of this file in the host file system is specified by the `riva_model_loc` parameter in the Riva configuration file `config.sh`under the Riva quickstart root folder.

### How to modify the lexicon file

First, locate and make a copy of the lexicon file. For example:
```
cp /data/models/citrinet-ctc-decoder-cpu-streaming/1/lexicon.txt  decoding_lexicon.txt
```

Next, modify it to add the sentencepiece tokenizations for the word of interest. For example, one could add:
```
manu ▁ma n u
manu ▁man n n ew
manu ▁man n ew
manu ▁men n ew
manu ▁me n u
```
which are 5 different pronunciations/tokenizations of the word `manu`.  If the acoustic model predicts those tokens, they will be decoded as `manu`.

Finally, once this is done, regenerate the model repository using that new decoding lexicon tokenization by passing `--decoding_lexicon=decoding_lexicon.txt` to `riva-build` instead of `--decoding_vocab=decoding_vocab.txt`.

### How to generate the tokenized form

When modifying the lexicon file, you’ll need to ensure that:

- The new lines follow the indentation/space pattern like the rest of the file and that the tokens used are part of the tokenizer model. 

- The tokens are valid tokens as determined by the tokernizer (packaged with all Riva acoustic model).

The latter ensure that you use only tokens that the acoustic model has been trained on. To do this, you’ll need the tokenizer model and the `sentencepiece` python package (`pip install sentencepiece`). You can get the tokenizer model for the deployed pipeline from the model repository ctc decoder directory for your model. It will be named `<hash>_tokenizer.model`.

You can then generate new lexicon entries with a command like this:
    

In [ ]:
!pip install sentencepiece

In [ ]:
!PRONUNCIATION="b raf"; TOKEN="BRAF"; echo "$PRONUNCIATION" | \
    spm_encode --model=[hash]_tokenizer.model --output_format=nbest_piece --nbest_size=4 | \
    sed "s/^/$TOKEN\t/"

Example output:
```
BRAF    ▁b ▁ra f
BRAF    ▁b ▁ ra f
BRAF    ▁ b ▁ra f
BRAF    ▁b ▁r a f
```

As sentencepiece is a probabilistic tokernizer, it can generate alternative tokernized form for the same words.